# Trabalho da disciplina de **LUI**

## O objetivo do trabalho é extrair informações de uma fonte de sua própria escolha, sendo através de uma **API** ou por **Web Scraping**.

## Você deve fornecer o código com o output do dataframe e as saidas após as seguintes operações de Ordenação e Filtragem.

## Etapas:

1.   Escolher uma API ou site que contenham as informações desejadas
    *   Filmes (API - https://www.omdbapi.com/) ou (Scraping - https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250)
    *   Alimentos (API - https://developer.nutritionix.com/docs/v1_1) ou (Scraping - https://www.nutritionix.com/database/common-foods)
    *   Tempo (API - https://openweathermap.org/api)
    *   Commodities (Scraping - https://www.indexmundi.com/commodities/?commodity=agricultural-raw-materials-price-index&months=360)
    *   Finanças (
Se você desejar também pode pesquisar sua própria API nesse site (https://the-api-collective.com/).

Acima estão listadas apenas sugestões, fiquem à vontade! Inclusive podem utilizar alguma apresentada em aula!

2.   Guardar todas essas informações extraídas em um DataFrame
3.   Executar a operação de ordenação em um atributo escolhido (ao seu critério, podendo ser em ordem numérica ou alfabética)
4.   Executar a operação de filtragem em um atributo escolhido (ao seu critério)


O trabalho pode conter até 5~6 pessoas por grupo, não mais do que isso!



****** LEIAM ATENTAMENTE A DOCUMENTAÇÃO DA SUA API ESCOLHIDA ******

Dúvidas?

prof.evelyn@ica.ele.puc-rio.br

prof.felipe@ica.ele.puc-rio.br

## Se a escolha for API ...

In [1]:
import requests
import time
import re
import nltk

import pandas as pd
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words("portuguese")

[nltk_data] Downloading package punkt to /home/rodmonte/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodmonte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
legislaturas = []
root = { 
        'dados': [], 
        'links': [
                    {
                        'rel': 'next', 
                        'href': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas?ordem=DESC&ordenarPor=id'
                    },
        ]
}

lastUrl = ''
while True:
    dados = root.get('dados', [])
    if dados:
        legislaturas += dados
    links = root.get('links', [])        
    for l in links:
        if l.get('rel','') == 'next':
            currUrl = l.get('href')
            break

    if currUrl == lastUrl:
        break
    else:
        root = requests.get(currUrl).json()
        lastUrl = currUrl           

In [3]:
legislaturas

[{'id': 56,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/56',
  'dataInicio': '2019-02-01',
  'dataFim': '2023-01-31'},
 {'id': 55,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/55',
  'dataInicio': '2015-02-01',
  'dataFim': '2019-01-31'},
 {'id': 54,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/54',
  'dataInicio': '2011-02-01',
  'dataFim': '2015-01-31'},
 {'id': 53,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/53',
  'dataInicio': '2007-02-01',
  'dataFim': '2011-01-31'},
 {'id': 52,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/52',
  'dataInicio': '2003-02-01',
  'dataFim': '2007-01-31'},
 {'id': 51,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/51',
  'dataInicio': '1999-02-01',
  'dataFim': '2003-01-31'},
 {'id': 50,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/legislaturas/50',
  'dataInicio': '1995-02-01',
  'dataFim': '1999-01-31'},
 {'id': 49,
  'uri':

In [4]:
deputados_ids = []
for legislatura in legislaturas[:1]:
    root = { 
        'dados': [], 
        'links': [
                    {
                        'rel': 'next', 
                        'href': f'https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome&idLegislatura={legislatura.get("id")}'
                    },
        ]
    }

    lastUrl = ''
    while True:      
        dados = root.get('dados', [])
        if dados:
            deputados_ids += [dep.get('id') for dep in dados]

        links = root.get('links', [])
        for l in links:
            if l.get('rel','') == 'next':
                currUrl = l.get('href')
                break

        if currUrl == lastUrl:
            break
        else:
            root = requests.get(currUrl).json()
            lastUrl = currUrl           

In [5]:
deputados = pd.DataFrame()
for deputado_id in deputados_ids:
    currUrl = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}'
    root = None
    while root is None:
        try:
            root = requests.get(currUrl).json()
        except Exception as e:
            print('Ocorreu um erro: ', e)
            time.sleep(0.1)

    dados = root.get('dados', {})
    dados['idLegislatura'] = dados.get('ultimoStatus',{}).get('idLegislatura',)
    dados['nome'] = dados.get('ultimoStatus',{}).get('nome',)
    dados['siglaPartido'] = dados.get('ultimoStatus',{}).get('siglaPartido',)
    
    for k, v in dados.items():
        dados[k] = [v]
        
    deputados = deputados.append(pd.DataFrame.from_dict(dados))

In [6]:
deputados.head()

,id,uri,nomeCivil,ultimoStatus,cpf,sexo,urlWebsite,redeSocial,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento,escolaridade,idLegislatura,nome,siglaPartido
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,JOSE ABILIO SILVA DE SANTANA,"{'id': 204554, 'uri': 'https://dadosabertos.ca...",36607606504,M,None,[],1965-02-13,None,BA,Salvador,Superior Incompleto,56,Abílio Santana,PL
0,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,PAULO SERGIO ABOU ANNI,"{'id': 204521, 'uri': 'https://dadosabertos.ca...",08496582841,M,None,"[https://www.facebook.com/abou.anni.7, https:/...",1966-11-06,None,SP,São Paulo,Superior,56,Abou Anni,PSL
0,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,ACÁCIO DA SILVA FAVACHO NETO,"{'id': 204379, 'uri': 'https://dadosabertos.ca...",74287028287,M,None,"[acaciofavacho, deputadofederalacaciofavacho, ...",1983-09-28,None,AP,Macapá,Superior,56,Acácio Favacho,PROS
0,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,ADOLFO VIANA DE CASTRO NETO,"{'id': 204560, 'uri': 'https://dadosabertos.ca...",80123848504,M,None,[],1981-02-02,None,BA,Salvador,Superior,56,Adolfo Viana,PSDB
0,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,ADRIANA MIGUEL VENTURA,"{'id': 204528, 'uri': 'https://dadosabertos.ca...",12519851813,F,None,"[adriventurasp, adriventurasp, adriventurasp, ...",1969-03-06,None,SP,São Paulo,Doutorado,56,Adriana Ventura,NOVO


In [7]:
deputados.nome = deputados.nome.map(str.title)
deputados.nomeCivil = deputados.nomeCivil.map(str.title)
deputados.head()

,id,uri,nomeCivil,ultimoStatus,cpf,sexo,urlWebsite,redeSocial,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento,escolaridade,idLegislatura,nome,siglaPartido
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Jose Abilio Silva De Santana,"{'id': 204554, 'uri': 'https://dadosabertos.ca...",36607606504,M,None,[],1965-02-13,None,BA,Salvador,Superior Incompleto,56,Abílio Santana,PL
0,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Paulo Sergio Abou Anni,"{'id': 204521, 'uri': 'https://dadosabertos.ca...",08496582841,M,None,"[https://www.facebook.com/abou.anni.7, https:/...",1966-11-06,None,SP,São Paulo,Superior,56,Abou Anni,PSL
0,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Da Silva Favacho Neto,"{'id': 204379, 'uri': 'https://dadosabertos.ca...",74287028287,M,None,"[acaciofavacho, deputadofederalacaciofavacho, ...",1983-09-28,None,AP,Macapá,Superior,56,Acácio Favacho,PROS
0,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana De Castro Neto,"{'id': 204560, 'uri': 'https://dadosabertos.ca...",80123848504,M,None,[],1981-02-02,None,BA,Salvador,Superior,56,Adolfo Viana,PSDB
0,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Miguel Ventura,"{'id': 204528, 'uri': 'https://dadosabertos.ca...",12519851813,F,None,"[adriventurasp, adriventurasp, adriventurasp, ...",1969-03-06,None,SP,São Paulo,Doutorado,56,Adriana Ventura,NOVO


In [8]:
discursos = pd.DataFrame()

STOPWORDS += [
    'de', 'da', 'das', 'do', 'dos', 'sr', 'sra', 'srs', 'sras', 'que', 'exa', 'uma', 'um', 'no', 'na', 'em', 'nos', 'nas', 'às', 
    'essa', 'esta', 'aquela', 'esse', 'este', 'aquele', 'isso', 'isto', 'aquilo',
    'art', 
]

for index, deputado in deputados.iterrows():

    currUrl = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado.id}/discursos'
    root = None
    while root is None:
        try:
            root = requests.get(currUrl).json()
        except Exception as e:
            print('Ocorreu um erro: ', e)
            time.sleep(0.1)

    dados = root.get('dados', {})
    qtd_docs = len(dados)
    dados = ' '.join([d['transcricao'] for d in dados])
    dados = dados.replace('\n',' ').replace('\r',' ').lower()
    dados = re.sub(r'[^a-z|^áéíóúàèìòùâêîôûãẽĩõũäëïöüç|^\s]', ' ', dados)
    dados = [t for t in nltk.word_tokenize(dados) if len(t) > 1 and t not in STOPWORDS]
    qtd_tokens = len(dados)
    dados = ' '.join(dados)
    
    dados = {
        'id': [deputado.id],
        'tokens': [dados],
        'qtd_tokens': [qtd_tokens],
        'qtd_docs': [qtd_docs],
    }           

    discursos = discursos.append(pd.DataFrame.from_dict(dados))

In [9]:
deputados = deputados.join(discursos.set_index('id'), on='id', how='inner')
del discursos

In [10]:
deputados[deputados.qtd_tokens > 0].sort_values(by=['qtd_tokens','qtd_docs'], ascending=False)[['nome','tokens','qtd_tokens','qtd_docs']]

,nome,tokens,qtd_tokens,qtd_docs
0,Nilto Tatto,nilto tatto pt sp revisão orador boa tarde pre...,1475,15
0,Pompeo De Mattos,pompeo mattos pdt rs revisão orador presidente...,1460,8
0,Zé Silva,zé silva solidariedade mg ordem revisão orador...,1380,11
0,Alessandro Molon,alessandro molon psb rj ordem revisão orador p...,1319,9
0,Ivan Valente,ivan valente psol sp revisão orador presidente...,1283,11
...,...,...,...,...
0,Igor Timo,igor timo pode mg ordem revisão orador preside...,64,1
0,Eduardo Bismarck,eduardo bismarck pdt ce ordem revisão orador p...,59,1
0,Franco Cartafina,franco cartafina pp mg ordem revisão orador pr...,55,1
0,Pastor Eurico,pastor eurico patriota pe revisão orador presi...,44,1


In [11]:
deputados[deputados.tokens.str.contains('genocida')][['nome','siglaPartido']]

,nome,siglaPartido
0,Bohn Gass,PT
0,Erika Kokay,PT
0,Helder Salomão,PT
0,Joseildo Ramos,PT
0,Luiza Erundina,PSOL
0,Marcon,PT
0,Paulão,PT
0,Rogério Correia,PT
0,Valmir Assunção,PT
0,Vivi Reis,PSOL
